In [144]:
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import torch


# sentences - документы с базы данных, пока несколько "документов", что я сам придумал, затем они прогоняются через эмбеддер

In [145]:
sentences = ["Банк России - национальный банк, учрежден в 1992 году.", "Деньги лучше класть в банк, чем в банку.", "Курс валют на сегодня - доллар 36 рублей, евро - 37.", "Лучший банк - цетральный банк."]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.04468967 -0.0918605  -0.00948589 ... -0.03272169  0.01700571
   0.111098  ]
 [ 0.05723943  0.15294039 -0.0171421  ...  0.11154366  0.0364729
   0.14252166]
 [-0.03234731 -0.47003657 -0.00886906 ... -0.0270731  -0.04644873
   0.02242003]
 [-0.0325066  -0.06682001 -0.0177744  ...  0.08461011 -0.01872939
   0.06934763]]


In [32]:
embeddings.shape

(4, 768)

In [57]:
# text_vectors = np.vstack(embeddings)
# text_vectors

# Понижение размерности с помощью PCA, пока до 2 компонентов на тест, лучше до ~300, когда будет большая база

In [33]:
pca = Pipeline(steps=[
    ('mean', StandardScaler(with_mean=True, with_std=False)),
    ('pca', PCA(n_components=2, random_state=42)),
    ('std', StandardScaler(with_mean=True, with_std=True))
])
pca.fit(embeddings)

Pipeline(steps=[('mean', StandardScaler(with_std=False)),
                ('pca', PCA(n_components=2, random_state=42)),
                ('std', StandardScaler())])

In [36]:
text_vectors = pca.transform(embeddings)
text_vectors.shape

(4, 2)

In [37]:
text_vectors

array([[ 0.1321801 ,  1.6664053 ],
       [-0.8906605 , -0.8592002 ],
       [ 1.5837332 , -0.68562907],
       [-0.8252529 , -0.1215768 ]], dtype=float32)

# Создаем annoy_index для того, чтобы потом проводить поиск и находить ближайших соседей

In [41]:
feature_vector_size = text_vectors.shape[1]
annoy_index = AnnoyIndex(feature_vector_size, 'angular')

In [42]:
text_collection = []

In [48]:
for i in range(text_vectors.shape[0]):
    annoy_index.add_item(len(text_collection), text_vectors[i])
    text_collection.append(sentences[i])

In [50]:
annoy_index.build(n_trees=2, n_jobs=-1)

True

In [52]:
annoy_index.save('test_cbr.ann')


True

# Загружаем языковую модель и токенизатор

In [59]:
llm_dirname = '../nsu-ai/team_code/models/llm'


In [60]:
llm_model = AutoModelForCausalLM.from_pretrained(llm_dirname, torch_dtype=torch.float16, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(llm_dirname)



Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.51s/it]


In [62]:
model = model.to('cuda') # решил закинуть эмбеддер на куду

In [63]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

# Берем вопрос и прогоняем через эмбеддер, понижаем размерность

In [121]:
sentence = ["Когда был основан Банк России?"]
embedding = model.encode(sentence)
pca_embedding = pca.transform(embedding)


In [122]:
pca_embedding = np.squeeze(pca_embedding)
pca_embedding.shape

(2,)

# находим 

In [123]:
found_indices = annoy_index.get_nns_by_vector(pca_embedding, n=1, search_k=-1)


In [124]:
found_indices

[0]

In [125]:
found_texts = [sentences[idx] for idx in found_indices]
found_texts

['Банк России - национальный банк, учрежден в 1992 году.']

In [128]:
full_prompt = '<s>[INST] вы полезный помощник банка россии, который вежливо отвечает на вопросы. ты знаешь что: ' + found_texts[0] + ' Ответьте согласно этой информации. ' + sentence[0] + ' [/INST]'

In [129]:
full_prompt

'<s>[INST] вы полезный помощник банка россии, который вежливо отвечает на вопросы. ты знаешь что: Банк России - национальный банк, учрежден в 1992 году. Ответьте согласно этой информации. Когда был основан Банк России? [/INST]'

In [142]:
model_input = tokenizer(full_prompt, return_tensors="pt").to("cuda")
input_prompt = tokenizer.batch_decode(torch.tensor(model_input['input_ids']), skip_special_tokens=True)[0]

with torch.no_grad():
    generated_text = tokenizer.decode(llm_model.generate(**model_input, max_new_tokens=1000)[0], skip_special_tokens=True)
    
answer = ' '.join(generated_text[len(input_prompt):].split()).strip()

/tmp/ipykernel_16180/1589065902.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_prompt = tokenizer.batch_decode(torch.tensor(model_input['input_ids']), skip_special_tokens=True)[0]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [143]:
answer

'Банк России был основан в 1992 году.'

/tmp/ipykernel_16180/1877989193.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_prompt = tokenizer.batch_decode(torch.tensor(model_input['input_ids']), skip_special_tokens=True)[0]
